In [1]:
from bs4 import BeautifulSoup #スクレイピングのためのライブラリ
import requests #HTTP操作用
import re
import time
import pandas as pd
import numpy as np
import sqlite3

/Users/takahashikaisei/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [ ]:
def createDB(year, month, )

日数をカウントする関数を定義する

In [52]:
def day_num_count(y, m):
    if m in [1, 3, 5, 7, 8, 10, 12]:
        day_num = 31
    elif m in [4, 6, 9, 11]:
        day_num = 30
    elif m == 2:    
        if y % 400 == 0:
            day_num = 29
        elif y % 100 == 0:
            day_num = 28
        elif y % 4 == 0:
            day_num = 29
        else:
            day_num = 28
    
    return day_num

In [ ]:
year = 2023
month = 12

#スクレイピングする部分
url = f'https://www.data.jma.go.jp/obd/stats/etrn/view/daily_a1.php?prec_no=44&block_no=0370&year={year}&month={month}'
r = requests.get(url)
soup = BeautifulSoup(r.content, 'html.parser')
tags = soup.find_all('td', class_='data_0_0')

#データをリストにまとめる部分
data = []
for i in tags:
    data.append(i.string)

#日数をカウントする部分
day_num_count(year, month)

#リストを日数分区切る部分
final_list = []
final_list = np.array_split(data, day_num) #day_numの数に区切ればいい

# データフレームに変換する部分
df = pd.DataFrame(final_list, columns=[
    "合計降水量",
    "最大一時間降水量",
    "最大10分間降水量",
    "平均気温",
    "最高気温",
    "最低気温",
    "平均湿度",
    "最小湿度",
    "平均風速",
    "最大風速",
    "最大風速の風向き",
    "最大瞬間風速",
    "最大瞬間風速の風向き",
    "最多風向き",
    "日照時間",
    "降雪の深さの合計",
    "最深積雪"
    ])


#日にちのリストを作る
day = []
for i in range(1, day_num+1):
    day.append(i)

#日付のリストを作る
hiduke = []
for tasu in day:
    hiduke.append(str(month) + '/' + str(tasu))

#シリーズ型に日付を格納
s = pd.Series(hiduke, name='date')

#データフレームに格納
df.insert(0, '日付', s)



con = sqlite3.connect('weather_scrape.sqlite') #データベースにアクセス

df.to_sql('weather_scrape_data', con, if_exists='replace', index=False) #データベースに変換、if_exists='replace'で同じ名前のものがあったときに置き換える。index=Falseでindexを入れないように指定してる

con.close() #データベースを閉じる